<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/GPTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 환경설정

In [12]:
import requests_html

In [11]:
pip install requests-html

In [ ]:
pip install lxml_html_clean

In [3]:
import requests
import json
import pandas as pd
import urllib.parse
import urllib3
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

In [19]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

In [5]:
class NewTLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")  # 원하는 Cipher Suite 설정
        kwargs["ssl_context"] = ctx
        return super().init_poolmanager(*args, **kwargs)

# 1. 저축은행

## 1.1. 신용점수별 금리

In [6]:
def savings_cs(year, month):
  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  data = response.text

  json_data = json.loads(data)

  # 데이터 추출 및 DataFrame 생성
  df = pd.DataFrame(json_data['REC'])
  df = df[['BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
            'A_RATE3', 'A_RATE_AVE']]

  return df


crawled_data = savings_cs(2024, 12)

crawled_data

,BANK_NAME,SUBMIT_MONTH,A_RATE1_3,A_RATE1,A_RATE2,A_RATE3,A_RATE_AVE
0,BNK,202412,12.97,13.7,14.83,15.13,14.31
1,DB,202412,11.55,11.71,12.1,NaN,11.91
2,IBK,202412,12.63,14.27,16.01,NaN,15.29
3,JT,202412,13.57,13.86,14.04,13.83,13.97
4,JT친애,202412,13.92,14.86,15.22,15.3,15.17
5,KB,202412,12.21,13.2,14.68,16.19,14.27
6,NH,202412,11.13,11.64,13.02,13.52,12.41
7,OK,202412,14.21,15.05,16.75,17.94,16.46
8,OSB,202412,14.73,15.42,16.11,18.01,15.93
9,SBI,202412,14.05,14.7,15.15,15.86,15.14


## 1.2. 금리대별 취급비중

In [ ]:
def savings_int(year, month):
    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    response = requests.post(url, data={"_JSON_": json.dumps(payload)})
    response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    df = pd.DataFrame(data["REC"])  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    desired_columns = ["BANK_NAME", "SUBMIT_MONTH", "HANDING_WEIGHT_10", "HANDING_WEIGHT_12", "HANDING_WEIGHT_14"
                        ,"HANDING_WEIGHT_16", "HANDING_WEIGHT_18", "HANDING_WEIGHT_20"]
    df = df[desired_columns] #원하는 열만 남기기

    return df

# 예시 사용

data = savings_int(2024, 12)

data

,BANK_NAME,SUBMIT_MONTH,HANDING_WEIGHT_10,HANDING_WEIGHT_12,HANDING_WEIGHT_14,HANDING_WEIGHT_16,HANDING_WEIGHT_18,HANDING_WEIGHT_20
0,BNK,202412,0.44,16.82,25.54,46.61,10.59,0
1,DB,202412,NaN,66.42,28.2,3.91,1.47,NaN
2,IBK,202412,NaN,9.23,36.05,14.95,12.6,27.17
3,JT,202412,35.07,17,36.23,6.88,4.82,NaN
4,JT친애,202412,NaN,NaN,33.06,42.01,16.68,8.25
5,KB,202412,5.18,24.26,24.11,16.61,12.85,16.99
6,NH,202412,0.09,44.58,32.35,18.56,3.38,1.04
7,OK,202412,NaN,5.68,9.36,27.59,28.89,28.48
8,OSB,202412,NaN,NaN,23.81,33.23,11.72,31.24
9,SBI,202412,0.54,8.64,24.48,33.08,19.18,14.08


# 2. 신용카드

## 2.1. 카드론_신용점수별 금리

In [11]:
def card_loan_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail25Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1458,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgCardPoint1", "cgCardPoint2", "cgCardPoint3"
                ,"cgCardPoint4", "cgCardPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    return df

# 예시 사용

data = card_loan_cs(2024, 11)

data

,mcCompany,cgcSeq,cgCardPoint1,cgCardPoint2,cgCardPoint3,cgCardPoint4,cgCardPointAvg
0,경남은행,202411,8.08,12.20,15.20,17.46,12.20
1,광주은행,202411,12.59,14.04,15.60,16.92,14.94
2,롯데카드,202411,11.10,12.86,15.96,18.02,14.93
3,부산은행,202411,11.04,12.11,14.01,15.59,12.33
4,비씨카드,202411,11.79,13.19,14.78,14.96,13.30
5,삼성카드,202411,12.25,13.04,15.51,18.00,14.79
6,신한카드,202411,11.09,12.78,15.00,17.19,14.46
7,씨티은행,202411,8.89,11.48,12.45,14.34,12.09
8,우리카드,202411,10.07,13.72,16.51,18.83,15.39
9,전북은행,202411,0.00,0.00,14.54,17.77,14.64


## 2.2. 현금서비스_신용점수별 금리

In [ ]:
def card_cash_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail20Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1460,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgMoneyPoint1", "cgMoneyPoint2", "cgMoneyPoint3"
                ,"cgMoneyPoint4", "cgMoneyPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    return df

# 예시 사용

data = card_cash_cs(2024, 11)

data

,mcCompany,cgcSeq,cgMoneyPoint1,cgMoneyPoint2,cgMoneyPoint3,cgMoneyPoint4,cgMoneyPointAvg
0,경남은행,202411,14.23,16.88,17.72,18.37,17.81
1,광주은행,202411,16.27,17.10,17.78,18.72,18.33
2,롯데카드,202411,15.30,16.50,17.96,19.16,18.50
3,부산은행,202411,16.34,17.48,18.26,18.37,17.94
4,비씨카드,202411,14.76,16.55,18.23,19.33,18.44
5,삼성카드,202411,14.33,15.25,17.37,19.01,17.81
6,신한카드,202411,13.75,15.08,16.95,18.67,17.66
7,씨티은행,202411,13.02,17.66,19.58,19.85,19.51
8,우리카드,202411,16.09,17.25,18.81,19.45,18.92
9,전북은행,202411,14.29,17.04,18.71,19.31,18.82


# 3. 캐피탈

## 3.1. 신용점수별 금리

In [ ]:
def capital_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditloan/creditloanDisclosureDetail11/ajax"

    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": 521,
          "clgcYyyy": 2024
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['clgcquarter'] == month:
            clgc_seq = item['clgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": 11,
          "cardItem": "134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64",
          "clgcSeq": clgc_seq,
          "clgcYyyy": year
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "clgcSeq", "clgPoint1", "clgPoint2", "clgPoint3"
                ,"clgPoint4", "clgPointAvg"]] #원하는 열만 남기기

    df["clgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    return df

capital_cs(2024, 12)


,mcCompany,clgcSeq,clgPoint1,clgPoint2,clgPoint3,clgPoint4,clgPointAvg
0,아이엠캐피탈,202412,12.07,13.13,14.90,16.02,14.85
1,오케이캐피탈,202412,17.99,19.99,19.99,19.99,19.99
2,한국캐피탈,202412,16.32,17.67,18.34,18.59,18.14
3,한국투자캐피탈,202412,12.92,13.62,14.60,16.10,14.81
4,BNK캐피탈,202412,12.73,13.19,13.41,13.94,13.44
5,KB캐피탈,202412,11.98,13.08,15.21,16.20,13.47
6,롯데캐피탈,202412,11.50,13.18,14.96,16.63,14.42
7,메리츠캐피탈,202412,0.00,17.61,19.71,19.83,19.61
8,우리금융캐피탈,202412,12.51,13.27,14.30,14.86,13.59
9,하나캐피탈,202412,11.47,11.62,12.44,13.06,11.95


# 4. 은행

## 4.1. 신용점수별 금리

In [21]:
import requests
from urllib.parse import urlencode

def crawl_kfb_loan_data(year, month, opt_1, detail, str_value, select_new_balance):
    """
    한국금융복지센터 대출 비교 정보를 크롤링하는 함수

    Args:
        year (str): 년도
        month (str): 월
        opt_1 (str): 옵션 1
        detail (str): 상세 정보
        str_value (str): 금융기관 코드 문자열
        select_new_balance (str): 잔액 기준

    Returns:
        str: 크롤링된 데이터 (성공 시) 또는 에러 메시지 (실패 시)
    """

    url = "https://portal.kfb.or.kr/compare/loan_household_new.php"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Cookie": "_fwb=212unpbFH4gShFqwzNvDsjk.1721955719107; _ga=GA1.3.1753461068.1722214698; _ga_0HWLZE2375=GS1.3.1722214698.1.1.1722214698.0.0.0; PHPSESSID=c1ek78m0dlg8oce33ud2t9fg74", # 실제 Cookie 값으로 변경 필요
        "Host": "portal.kfb.or.kr",
        "Origin": "https://portal.kfb.or.kr",
        "Referer": "https://portal.kfb.or.kr/compare/loan_household_new.php",
        "Sec-Ch-Ua": '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Windows"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }

    payload = {
        "year": year,
        "month": month,
        "opt_1": opt_1,
        "detail": detail,
        "str": str_value,
        "select_new_balance": select_new_balance
    }

    with requests.session() as s:
      s.mount("https://", TLSAdapter())
      response = s.post(url, headers=headers, data=payload)
      response.raise_for_status()
    return response.text

# 사용 예시
year = "2025"
month = "01"
opt_1 = "3"
detail = "0"
str_value = "KDB%25BB%25EA%25BE%25F7%25C0%25BA%25C7%25E0%7CNH%25B3%25F3%25C7%25F9%25C0%25BA%25C7%25E0%7C%25BD%25C5%25C7%25D1%25C0%25BA%25C7%25E0%7C%25BF%25EC%25B8%25AE%25C0%25BA%25C7%25E0%7CSC%25C1%25A6%25C0%25CF%25C0%25BA%25C7%25E0%7C%25C7%25CF%25B3%25AA%25C0%25BA%25C7%25E0%7CIBK%25B1%25E2%25BE%25F7%25C0%25BA%25C7%25E0%7CKB%25B1%25B9%25B9%25CE%25C0%25BA%25C7%25E0%7C%25C7%25D1%25B1%25B9%25BE%25BE%25C6%25BC%25C0%25BA%25C7%25E0%7CSh%25BC%25F6%25C7%25F9%25C0%25BA%25C7%25E0%7CiM%25B9%25F0%25C5%25A9%2528%25B1%25B8%2B%25B4%25EB%25B1%25B8%25C0%25BA%25C7%25E0%2529%7CBNK%25BA%25CE%25BB%25EA%25C0%25BA%25C7%25E0%7C%25B1%25A4%25C1%25D6%25C0%25BA%25C7%25E0%7C%25C1%25A6%25C1%25D6%25C0%25BA%25C7%25E0%7C%25C0%25FC%25BA%25CF%25C0%25BA%25C7%25E0%7CBNK%25B0%25E6%25B3%25B2%25C0%25BA%25C7%25E0%7C%25C4%25C9%25C0%25CC%25B9%25F0%25C5%25A9%7C%25C4%25AB%25C4%25AB%25BF%25C0%25B9%25F0%25C5%25A9%7C%25C5%25E4%25BD%25BA%25B9%25F0%25C5%25A9"
select_new_balance = "1"

result = crawl_kfb_loan_data(year, month, opt_1, detail, str_value, select_new_balance)
print(result)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">
<head>
<title>전국은행연합회 소비자포털</title>
<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta http-equiv="imagetoolbar" content="no" />
<meta name="viewport" content="width=1200" />
<meta name="apple-mobile-web-app-title" content="전국은행연합회 소비자포털" />
<meta name="robots" content="index,follow"/>
<!-- <meta name="naver-site-verification" content="61315e811a24bc961d6d9b08f8032dcdeb0df528" /> -->
<meta name="description" content="금융상품정보, 금리/수수료 비교공시, 금융서비스정보, 소비자정보, 금융교육, 보이스피싱정보"/>
<meta property="og:title" content="전국은행연합회 소비자포털" />
<meta property="og:url" content="https://portal.kfb.or.kr" />
<meta property="og:image" content="https://portal.kfb.or.kr/images/kfb_icon.png" />
<meta property="og:description" content="금융상품정보, 금리/수수

In [25]:
import asyncio
import requests
from urllib.parse import urlencode
from requests_html import AsyncHTMLSession
from requests.adapters import HTTPAdapter
import ssl
import nest_asyncio
nest_asyncio.apply()

class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")  # 또는 더 안전한 Cipher Suite 지정
        kwargs["ssl_context"] = ctx
        return super().init_poolmanager(*args, **kwargs)

async def crawl_kfb_loan_data(year, month, opt_1, detail, str_value, select_new_balance):
    url = "https://portal.kfb.or.kr/compare/loan_household_new.php"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Cookie": "_fwb=212unpbFH4gShFqwzNvDsjk.1721955719107; _ga=GA1.3.1753461068.1722214698; _ga_0HWLZE2375=GS1.3.1722214698.1.1.1722214698.0.0.0; PHPSESSID=c1ek78m0dlg8oce33ud2t9fg74",  # 실제 Cookie 값으로 변경 필요
        "Host": "portal.kfb.or.kr",
        "Origin": "https://portal.kfb.or.kr",
        "Referer": "https://portal.kfb.or.kr/compare/loan_household_new.php",
        "Sec-Ch-Ua": '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Windows"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }

    payload = {
        "year": year,
        "month": month,
        "opt_1": opt_1,
        "detail": detail,
        "str": str_value,
        "select_new_balance": select_new_balance
    }

    try:
        session = AsyncHTMLSession()
        with session as s:
            s.mount("https://", TLSAdapter())
            r = await s.post(url, headers=headers, data=payload)
            r.raise_for_status()

            # 1. arender()에 timeout 및 sleep 매개변수 사용
            await r.html.arender(timeout=20, sleep=5)  # 필요에 따라 timeout 및 sleep 값 조정

            result = await r.html.arender(script="""
                return document.querySelector('.Resultitle ul.info li.leftArea span.title').textContent;
            """)

            # 4. 특정 텍스트가 나타나는지 확인 (예: "대출금리" 텍스트)
            # if "대출금리" in r.html.html:
            #     print("Data loaded based on text check")
            #     # 이제 r.html.html 파싱 가능

            # 5. JavaScript 코드 실행 (고급)
            # await r.html.arender(script="""
            #     const element = document.querySelector('#someElement');
            #     if (element && element.textContent === 'Expected Value') {
            #         return true;
            #     }
            #     return false;
            # """, timeout=20, sleep=5)

            return r.html.html

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

async def main():  # main 함수를 async로 정의
    year = "2025"
    month = "01"
    opt_1 = "3"
    detail = "0"
    str_value = "KDB%BB%EA%BE%F7%C0%BA%C7%E0|NH%B3%F3%C7%F9%C0%BA%C7%E0|%BD%C5%C7%D1%C0%BA%C7%E0|%BF%EC%B8%AE%C0%BA%C7%E0|SC%C1%A6%C0%CF%C0%BA%C7%E0|%C7%CF%B3%AA%C0%BA%C7%E0|IBK%B1%E2%BE%F7%C0%BA%C7%E0|KB%B1%B9%B9%CE%C0%BA%C7%E0|%C7%D1%B1%B9%BE%BE%C6%BC%C0%BA%C7%E0|Sh%BC%F6%C7%F9%C0%BA%C7%E0|iM%B9%F0%C5%A9%28%B1%B8+%B4%EB%B1%B8%C0%BA%C7%E0%29|BNK%BA%CE%BB%EA%C0%BA%C7%E0|%B1%A4%C1%D6%C0%BA%C7%E0|%C1%A6%C1%D6%C0%BA%C7%E0|%C0%FC%BA%CF%C0%BA%C7%E0|BNK%B0%E6%B3%B2%C0%BA%C7%E0|%C4%C9%C0%CC%B9%F0%C5%A9|%C4%AB%C4%AB%BF%C0%B9%F0%C5%A9|%C5%E4%BD%BA%B9%F0%C5%A9"
    select_new_balance = "1"

    result = await crawl_kfb_loan_data(year, month, opt_1, detail, str_value, select_new_balance)  # await 필수

    if result:
        print(result)
    else:
        print("크롤링 실패")

asyncio.run(main())  # asyncio.run()으로 main 함수 실행

An unexpected error occurred: cannot access local variable 'row_data' where it is not associated with a value
크롤링 실패


In [21]:
from bs4 import BeautifulSoup


soup = BeautifulSoup(result, 'html.parser') # html.parser는 파이썬 내장 HTML 파서입니다. 필요에 따라 lxml, html5lib 등을 설치하여 사용할 수 있습니다.

# 파싱된 데이터에서 원하는 요소 추출
title = soup.title.string
h1 = soup.h1.string
p = soup.p.string

print(title)  # Output: Sample Page
print(h1)  # Output: Hello, World!
print(p)  # Output: This is a sample HTML page.

전국은행연합회 소비자포털
None
정보 이용시 유의사항 안내
